In [1]:
from pyspark.sql.functions import col, when, to_date, lit

# 1. Configuração para evitar erro de datas antigas
spark.conf.set("spark.sql.legacy.parquet.datetimeRebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.legacy.parquet.datetimeRebaseModeInWrite", "CORRECTED")

# Caminho físico do lakehouse de destino
path_destino = "abfss://ws_sistemas@onelake.dfs.fabric.microsoft.com/lk_systemmax.Lakehouse/Tables/tab_funcionarios"

df_resultado = spark.sql("""
SELECT
    f.CHAPA,
    f.NOME,
    fu.CODIGO    AS CODFUNCAO,
    fu.NOME      AS FUNCAO,
    f.CODSITUACAO,
    f.CODEQUIPE,
    eq.DESCRICAO AS EQUIPE,
    pe.DTNASCIMENTO,
    f.DATAADMISSAO
FROM pfunc   AS f
INNER JOIN pfuncao AS fu
  ON fu.CODCOLIGADA = f.CODCOLIGADA
 AND fu.CODIGO      = f.CODFUNCAO
INNER JOIN pequipe AS eq
  ON eq.CODINTERNO  = f.CODEQUIPE
INNER JOIN ppessoa AS pe
  ON pe.CODIGO      = f.CODPESSOA
WHERE f.CODSITUACAO <> 'D'
  AND f.CODCOLIGADA = 2
""")

# 3. Corrigir datas inválidas (< 1900-01-01)
data_minima = to_date(lit("1900-01-01"))
data_padrao = to_date(lit("2000-01-01"))

df_corrigido = df_resultado \
    .withColumn("DTNASCIMENTO", when(col("DTNASCIMENTO") < data_minima, data_padrao).otherwise(col("DTNASCIMENTO")))


# Salva no Lakehouse de destino usando caminho físico
df_corrigido.write.format("delta").mode("overwrite").save(path_destino)

StatementMeta(, 78d92426-bb51-4e35-a32c-007fa5d190d7, 3, Finished, Available, Finished)